# Finetuning des Parser

Hier soll eine Möglichkeit erarbeitet werden, mit der die HTML-Files am besten extrahiert und bereinigt werden können. Damit soll eine reibungslose Weiterverarbeitung der Texte ermöglicht werden.

## Vorüberlegung:

Für unser Projekt sind verschiedene Teile des HTML-Files besonders von Bedeutung:

- Text: Inhalt der HTML-Seite
    - Diese sollte wenn möglich in einzelne Abschnitte gegliedert werden (ermöglicht später die exakte Beantwortung von Fragen)
    - Unwichtige Elemente der Seite (wie Menü-Punkte, die auf jeder Seite erscheinen) gilt es zu filtern
    - Nichtdarstellbare Zeichen gilt es zu filtern
- Überschrift und Unterüberschrift der Paragraphen
    - Wenn man im späteren Verlauf auf eine Frage einen Paragrafen referenzieren möchte, ist es wichtig die zum Paragraph gehörende Unterüberschrift und die Verlinkung dahin zu vermerken
- HTML-File: Der Link zum verwendeten File sollte als Meta-Data vermerkt werden um später zur Seite navigieren zu können
- Links und Bilder: Da wir uns noch nicht im klaren sind ob die Verlinkungen oder dargestellten Bilder für unsere Zwecke nützlich werden könnten, sollten diese zur Vorsicht mit abgespeichert werden.

### Benötigte Pakete:

In [1]:
from bs4 import BeautifulSoup
import glob
import re
import csv

### Auslesen von HTML-Elementen

Hier wird zunächst nur repräsentativ ein kleiner Teil der Dokumentationsdateien ausgelesen (alle Seiten zum Punkt "Bedienung").
Die ausgelesenen Daten werden in einem CSV-File gespeichert:

|Titel|Unterkapitel|Body|Links|Bilder|

In [104]:
path = 'way/to/files/*.html'
files = glob.glob(path)
header = ['title','unterkapitel', 'body', 'links','bilder']
with open ('test_cleanFiles.csv','w',encoding='utf-8') as save:
    writer = csv.writer(save)
    writer.writerow(header)
    for file in files:
        f = open(file, 'r', encoding = "utf-8")
        soup = BeautifulSoup(f.read(),'html.parser')
        data_text = get_body(soup)
        all_unter = get_paraName(soup)
        all_link = get_link(soup)
        all_bild = get_pic(soup)
        data = [re.sub('\W+',' ',soup.h1.get_text()), all_unter, data_text, all_link, all_bild]
        #write the data
        writer.writerow(data)
        f.close()

['EasyInsert', 'Tastenkombinationen']
['Fernsteuern-von-Produktname', 'Kommandozeile-Batch-Schnittstelle', 'COM-API', 'REST-API', 'Jenkins-Plug-in']
['Filter', 'Einfache-Filter', 'Erweiterte-Filter']
['Hauptmenu', 'Datei', 'Bearbeiten', 'Ansicht', 'Fenster', 'Optionen', 'Extras', 'Hilfe']
['Bedienung']
['Automatische-Installation-von-Produktname']
['Patches']
['Suche']
['Tastenkombinationen']


<>:14: DeprecationWarning: invalid escape sequence '\W'
<>:14: DeprecationWarning: invalid escape sequence '\W'
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_39852\3876118319.py:14: DeprecationWarning: invalid escape sequence '\W'
  data = [re.sub('\W+',' ',soup.h1.get_text()), all_unter, data_text, all_link, all_bild]


#### Erzeugen des Text-Body Elements

Im Div-Block mit der Klasse "document" wird der Inhalt der Seite umschlossen (ausgeschlossen das Seiten Menü, Header und Fußzeile)

Die Unterteilungen in der Seite erfolgen durch kleinere Div-Blöcke der Klasse "section".
Die id der Div-Blöcke entspricht der Überschrift (h1 oder h2).
Daher werden nun Anhand der Überschriften zunächst die Div-Blöcke gesucht und anschließend die p-Blöcke ausgelesen.

In [73]:
def get_body(soup_elem):
    liste_h2 = get_paraName(soup_elem)
    clean_para=[]
    ordnen = []
    
    for h2 in liste_h2:
        block = soup_elem.find("div",{"id":h2.lower()})
        for para in block.find_all("p"):
            clean_para.append(re.sub("\n", " ", para.get_text().strip(), flags=re.M))
        ordnen.append((h2, clean_para))
    return ordnen

#### Finde Kapitel

Es müssen die Kapitelnamen und die Namen der Unterkapitel für jede Datei gesucht werden. Nur so kann später der Inhalt den Überschriften zugeordnet werden.
Ein zu der Überschrift gehörender Block, trägt meist den Namen der dazugehördenen Überschrift. Ausnahmen entstehen durch Sonderzeichen.

Überlegung: Zur Zeit werden die Namen der Überschriften noch umständlich angepasst, um als id der zugehörigen Div-Blöcke verwendet zu werden. Vielleicht gibt es da einen besseren Work-Around.

In [103]:
def get_paraName(soup_elem):
    all_unter = []
    heading_tags = ["h1", "h2"]
    for unterkap in soup_elem.find_all(heading_tags):
        sauber = re.sub('\uf0c1','', unterkap.get_text())
        string = sauber.replace("(","").replace(")","")
        sauber = re.sub('\W+',' ',string)
        sauber = re.sub('ü','u',sauber)
        sauber = sauber.replace("ECU TEST","Produktname")
        all_unter.append(re.sub(' ','-', sauber))
    return all_unter

<>:7: DeprecationWarning: invalid escape sequence '\W'
<>:7: DeprecationWarning: invalid escape sequence '\W'
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_39852\1664730906.py:7: DeprecationWarning: invalid escape sequence '\W'
  sauber = re.sub('\W+',' ',string)


#### Finde Links

In [6]:
def get_link(soup_elem):
    all_link = []
    for link in soup_elem.find_all('a'):
        all_link.append(link.get('href'))
    return all_link 

#### Finde Bilder

In [7]:
def get_pic(soup_elem):
    all_bild=[]
    for bild in soup_elem.find_all('img'):
        all_bild.append(bild.get('src'))
    return all_bild

### Test-Bereich für einzelnen Funktionen:

In [18]:
HTMLFile = open("way/to/Hauptmenue.html", "r", encoding = "utf-8")
index = HTMLFile.read()
S = BeautifulSoup(index,'html.parser')

Idee: Alle Paragrafen sind als Div Elemente durch die entsprechende Überschrift (h2) als ID versehen

    - Sammle alle h2-Überschriften einer Seite
    - Rufe anhand der Überschriften die zugehörigen h2 Divs auf
    - extrahiere aus ihnen die p-Blöcke mit dem eigentlichen Inhalte der Seite

In [105]:
liste_h2 = get_paraName(S)
print(liste_h2)

['Hauptmenu', 'Datei', 'Bearbeiten', 'Ansicht', 'Fenster', 'Optionen', 'Extras', 'Hilfe']


Für jedes Unterkapitel (h2-Überschrift) finde alle P-Blöcke und übernimm den bereinigten Textinhalt in eine Liste. Anschließend ordne diese Liste der entsprechenden Überschrift zu.

Aufbau Liste:

| Paragrafen   | Inhalt                               |
|:----------- | :---------------------------------- |
|Überschrift 1 | (Textblock p1), (Textblock p2), .... |
|Überschrift 2 | (Textblock p1), (textblock p2), .... |

In [ ]:
clean_para=[]
ordnen = []
for h2 in liste_h2:
    block = S.find("div",{"id":h2.lower()})
    for para in block.find_all("p"):
        clean_para.append(re.sub("\n", " ", para.get_text().strip(), flags=re.M))
    ordnen.append((h2, clean_para))
print(ordnen[2])
    

### Problem

- Aktuell wird für die Überschrift h1 noch der gesamte Seiteninhalt vermerkt -> man muss also überlegen wie man hier nur den Text bis zur ersten h2-Überschrift extrahieren kann